In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
import os
from dotenv import load_dotenv

import pymysql
# 상위 폴더(../)에 있는 `.env` 파일 로드
dotenv_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))
load_dotenv(dotenv_path)


True

In [2]:
## mysql 연결
os.getenv('user_id')
os.getenv('user_password')

mysql_url = f"jdbc:mysql://{os.getenv('host_ip')}:3306/{os.getenv('DATABASE')}?useSSL=false&allowPublicKeyRetrieval=true&useUnicode=true&characterEncoding=UTF-8"

In [3]:
# SparkSession 생성
spark = SparkSession.builder \
    .appName("HDFS to MySQL") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()


25/02/04 13:06:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# HDFS에 있는 모든 Parquet 파일 로드
df = spark.read.parquet("hdfs:///data/")
print(df.count())

136568


In [5]:
## 시간 형식 수정(yyyy.mm.dd HH:MM:SS)
convert_cols = ['comment_publish_date', 'publish_date']
for c in convert_cols:
    df = df.withColumn(c
                     , to_timestamp(col('comment_publish_date'), "yyyy-MM-dd'T'HH:mm:ss'Z'")
                     )

In [6]:
## 결측치 제거
df = df.dropna()

In [10]:
# PyMySQL로 MySQL 연결 (SQL 실행을 위해)
conn = pymysql.connect(
    host=os.getenv('host_ip'),
    user=os.getenv('user_id'),
    password=os.getenv('user_password'),
    database=os.getenv('DATABASE'),
)

cursor = conn.cursor()

# ✅ MySQL 테이블 `TRUNCATE`
truncate_query = "TRUNCATE TABLE tbCrawled_Youtube"
cursor.execute(truncate_query)

conn.commit()
cursor.close()
conn.close()


In [10]:
# os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars /home/ubuntu/mysql-connector-java-5.1.49/mysql-connector-java-5.1.49.jar pyspark-shell"
# os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars /usr/local/lib/mysql-connector-java-5.1.49/mysql-connector-java-5.1.49.jar pyspark-shell"
df.write.format('jdbc')\
    .options(
        url = mysql_url
        , driver = 'com.mysql.jdbc.Driver'
        , dbtable = 'tbCrawled_Youtube'
        , user = os.getenv('user_id')
        , password = os.getenv('user_password')
    )\
    .mode('append')\
    .save()

Py4JJavaError: An error occurred while calling o79.save.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:102)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:217)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:221)
	at org.apache.spark.sql.execution.da

In [12]:
spark.stop()

In [14]:
import os
from pyspark.sql import SparkSession

host_ip = '15.168.221.131'
DATABASE = 'sample'
user_id = 'lab13'
user_password = 'lab13'

# 환경변수 설정
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/lib/mysql-connector-java-5.1.49-bin.jar pyspark-shell'

ss = SparkSession\
            .builder\
            .appName('MySQL')\
            .config('spark.executor.memory', '6g')\
            .config('spark.driver.memory', '6g')\
            .getOrCreate()

mysql_url = f'jdbc:mysql://{host_ip}:3306/{DATABASE}?useSSL=false&allowPublicKeyRetrieval=true'
ss.read.format('jdbc')\
            .options(
                url = mysql_url
                , driver = 'com.mysql.jdbc.Driver'
                , dbtable = 'tbSample'
                , user = user_id
                , password = user_password
                    )\
        .load()\
        .show()

Py4JJavaError: An error occurred while calling o157.load.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:102)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:38)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:32)
	at org.apache.spark.sql.e